<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF-393 Aprendizaje Automático II-2020 </h1>
    <h3> Taller 1 </h3>
</center>

Nombres: Nicolás Rosas Gómez  & Camilo Núñez Fernández

Roles: 201573608-1 &201573573-5

Correos:
nicolas.rosasg@sansano.usm.cl & camilo.nunezf@sansano.usm.cl
<hr style="height:2px;border:none"/>

**Temas**  

* Clasificadores Discriminativos Clásicos
* Clasificadores Generativos Clásicos
* Regresión Lineal y Regularización 
* Selección de Características (parcial, seguiremos en T2)
* Reducción de Dimensionalidad


<hr style="height:2px;border:none"/>

# Pregunta de Investigación

## Pregunta seleccionada:
**Reducción de Dimensionalidad**: Es igual de efectivo aplicar PCA sobre variables numéricas continuas que aplicarlo sobre variables categóricas codificadas numéricamente.

In [1]:
from sklearn.datasets import make_multilabel_classification
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from time import time
import pandas as pd
import numpy as np

import os
try:
    import prince
except ImportError:
    os.system('pip install prince')
    import prince

## Desarrollo

La estrategia a desarrollar para afirmar o refutar la hipotesis definida anteriormente, constara de calcular en tres dataset distintos, la descomposicion PCA sobre sus variables continuas, categoricas y enteras de manera independiente. A continuacion se obtendrá el porcentaje de varianza y el `cumulative sum` de estos valores. Además se entrenará un modeló de regresion lineal para todos los casos, a partir del cual se comentará el MSE de los valores predichos en cada caso.

Se consideraron dos dataset sintéticos y dos reales.

## Abalone dataset (Real)

### Con PCA y sin columna categorica

Comenzamos cargando el dataset desde el archivo original:Comenzamos cargando el dataset desde el archivo original:

In [2]:
abalone_data = pd.read_csv("abalone.csv")

Limpiaremos el dataset, y definiremos el conjunto $Y$ como la varaible `Weight`.

In [3]:
categorical_data = abalone_data.select_dtypes("O")
X_abalone = abalone_data.loc[:, abalone_data.columns != 'Weight']
X_abalone = X_abalone.loc[:, X_abalone.columns != 'Sex']
y_abalone = abalone_data["Weight"]


A continuación asignamos el modelo de regresión para probar los resultados:

In [4]:
PCA_model = LinearRegression()

Aplicamos PCA y reducimos la dimencionalidad a 6 feature:

In [5]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X_abalone, y_abalone, test_size=0.25, random_state=0)


pca = PCA(n_components=6)
pca.fit(X_train, y_test)

## Train
PCA_X_train = pca.transform(X_train)

## Test
PCA_X_test = pca.transform(X_test)

CPU times: user 47.2 ms, sys: 153 ms, total: 200 ms
Wall time: 17.6 ms


Revisamos los valores procentuales de la varianza:

In [6]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[1.03807187e+01 7.31139961e-02 2.78958153e-03 2.01560681e-03
 9.78197249e-04 5.72483154e-04]
[9.92388314e-01 6.98963893e-03 2.66681739e-04 1.92690381e-04
 9.35148663e-05 5.47289267e-05]
[0.99238831 0.99937795 0.99964463 0.99983732 0.99993084 0.99998557]


Luego entrenamos el modelo de regresión:

In [7]:
PCA_model.fit(PCA_X_train, y_train)
pred_train = PCA_model.predict(PCA_X_train)
pred_test = PCA_model.predict(PCA_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.0022363715351909285
MSE_Test: 0.0017495303027569283


### Con PCA y con columna categorica (One Hot Encoding)

En este caso, connsideraremos un `One Hot Encoding` para las variables categóricas:

In [8]:
ohe = OneHotEncoder().fit(categorical_data)
categorical_bin = ohe.transform(categorical_data)
categorical_bin = pd.DataFrame(categorical_bin.toarray(), columns=ohe.get_feature_names())
X_abalone_cat = pd.concat([X_abalone,categorical_bin], axis=1, ignore_index=True)

Replicamos el modelo de regresión:

In [9]:
PCA_model_cat = LinearRegression()

Aplicamos la descomposición de PCA para este caso con `OHE`, para 6 componentes:

In [10]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X_abalone_cat, y_abalone, test_size=0.25, random_state=0)

pca = PCA(n_components=6)

pca.fit(X_train, y_test)
## Train
PCA_X_train = pca.transform(X_train)

## Test
PCA_X_test = pca.transform(X_test)

CPU times: user 123 ms, sys: 377 ms, total: 500 ms
Wall time: 23.4 ms


In [11]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[1.04432385e+01 3.46225907e-01 2.73879605e-01 5.54210557e-02
 2.76524270e-03 2.01319116e-03]
[9.38698834e-01 3.11207923e-02 2.46178872e-02 4.98156587e-03
 2.48556049e-04 1.80957296e-04]
[0.93869883 0.96981963 0.99443751 0.99941908 0.99966763 0.99984859]


In [12]:
PCA_model_cat.fit(PCA_X_train, y_train)
pred_train = PCA_model_cat.predict(PCA_X_train)
pred_test = PCA_model_cat.predict(PCA_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.002462558647550641
MSE_Test: 0.001933513535465762


### Con MCA y columna categorica (_prince package_)

En este caso, aplicaremos la descomposición del método `MCA` sobre el dataset con columnas categóricas:

In [13]:
MCA_prince_model = LinearRegression()
prince_mca = prince.MCA(n_components=6)

X_train, X_test, y_train, y_test = train_test_split(X_abalone_cat, y_abalone, test_size=0.25, random_state=0)

prince_mca = prince_mca.fit(X_train, y_train) 
prince_mca_X_train = prince_mca.transform(X_train) 

prince_mca_X_test = prince_mca.transform(X_test) 

MCA_prince_model.fit(prince_mca_X_train, y_train)

pred_train = MCA_prince_model.predict(prince_mca_X_train)
pred_test = MCA_prince_model.predict(prince_mca_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.012607759508113284
MSE_Test: 0.0117558193042838


### Con MCA y sin columna categorica (_prince package_)

Consideramos el mismo método pero ahora sin considerar las columnas categóricas:

In [14]:
MCA_prince_model = LinearRegression()
prince_mca = prince.MCA(n_components=6)

X_train, X_test, y_train, y_test = train_test_split(X_abalone, y_abalone, test_size=0.25, random_state=0)

prince_mca = prince_mca.fit(X_train, y_train) 
prince_mca_X_train = prince_mca.transform(X_train) 

prince_mca_X_test = prince_mca.transform(X_test) 

MCA_prince_model.fit(prince_mca_X_train, y_train)

pred_train = MCA_prince_model.predict(prince_mca_X_train)
pred_test = MCA_prince_model.predict(prince_mca_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.014424450091002051
MSE_Test: 0.014538565398065365


## Data Sintetica

Para el desarrollo de este test, se utilizará la funcion `make_multilabel_classification` de `sklearn`.

### Datos enteros sin OHE (One-Hot Encoding)

In [15]:
X, y = make_multilabel_classification(n_classes=2, n_labels=10,
                                      allow_unlabeled=True,
                                      random_state=1)


In [16]:
int_synt_model = LinearRegression()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

pca = PCA(n_components=10)
pca.fit(X_train, y_test)

## Train
PCA_X_train = pca.transform(X_train)

## Test
PCA_X_test = pca.transform(X_test)

In [18]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[10.37512632  8.2876105   5.04167236  4.96265839  4.71654274  3.70596353
  3.367316    3.22853737  2.77031121  2.44024777]
[0.17344275 0.13854539 0.08428249 0.0829616  0.07884724 0.06195322
 0.05629199 0.05397201 0.04631176 0.04079404]
[0.17344275 0.31198814 0.39627063 0.47923223 0.55807947 0.6200327
 0.67632469 0.7302967  0.77660846 0.81740249]


In [19]:
int_synt_model.fit(PCA_X_train, y_train)
pred_train = int_synt_model.predict(PCA_X_train)
pred_test = int_synt_model.predict(PCA_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.05408020025947395
MSE_Test: 0.043031595426522184


### Datos enteros Con OHE (One-Hot Encoding)

In [20]:
ohe = OneHotEncoder().fit(X)
categorical_bin = ohe.transform(X)
categorical_bin = pd.DataFrame(categorical_bin.toarray(), columns=ohe.get_feature_names())
categorical_bin

,x0_0.0,x0_1.0,x0_2.0,x0_3.0,x0_4.0,x1_0.0,x1_1.0,x1_2.0,x1_3.0,x2_0.0,...,x19_1.0,x19_2.0,x19_3.0,x19_4.0,x19_5.0,x19_6.0,x19_7.0,x19_8.0,x19_9.0,x19_11.0
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
enc_synt_model = LinearRegression()

In [22]:
X_train, X_test, y_train, y_test = train_test_split(categorical_bin, y, test_size=0.25, random_state=0)

pca = PCA(n_components=40)
pca.fit(X_train, y_test)

## Train
PCA_X_train = pca.transform(X_train)

## Test
PCA_X_test = pca.transform(X_test)

In [23]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[0.81662074 0.73503703 0.71059335 0.64127897 0.62736686 0.58346687
 0.53939481 0.51065531 0.47074867 0.45516285 0.43599507 0.43333791
 0.41754275 0.40676903 0.39359945 0.36989318 0.3579849  0.33044969
 0.30822247 0.29400282 0.28802564 0.27689049 0.26885454 0.26314535
 0.25168733 0.23789218 0.22599413 0.21548488 0.20455944 0.20136256
 0.18700903 0.18300426 0.167857   0.16119449 0.15534525 0.15196657
 0.14634839 0.13635782 0.13345737 0.12636677]
[0.05203377 0.04683538 0.04527787 0.04086127 0.03997481 0.03717757
 0.03436937 0.03253814 0.02999535 0.02900225 0.02778091 0.0276116
 0.02660516 0.02591867 0.02507953 0.023569   0.02281022 0.02105573
 0.01963944 0.01873339 0.01835253 0.01764302 0.01713098 0.0167672
 0.01603711 0.01515811 0.01439998 0.01373035 0.0130342  0.0128305
 0.01191592 0.01166074 0.01069558 0.01027105 0.00989835 0.00968307
 0.00932508 0.0086885  0.00850369 0.00805189]
[0.05203377 0.09886915 0.14414702 0.18500829 0.2249831  0.26216067
 0.29653005 0.32906819 0.35906354 0.3880

## Avocado dataset (Dataset Real - Mixed Dataset)

### Sin Variables categoricas

In [24]:
avocado_data = pd.read_csv("avocado.csv")
categorical_data = avocado_data.select_dtypes("O").drop("Date", axis=1)
X_avocado = avocado_data.loc[:, avocado_data.columns != 'Date']
X_avocado = X_avocado.loc[:, X_avocado.columns != 'AveragePrice']
X_avocado = X_avocado.loc[:, X_avocado.columns != 'year']
X_avocado = X_avocado.loc[:, X_avocado.columns != 'type']
X_avocado = X_avocado.loc[:, X_avocado.columns != 'region']
y_avocado = avocado_data["AveragePrice"]
X_avocado

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags
0,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0
1,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0
2,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0
3,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0
4,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0
...,...,...,...,...,...,...,...,...
18244,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0
18245,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0
18246,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0
18247,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0


In [25]:
PCA_model_avocado = LinearRegression()

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_avocado, y_avocado, test_size=0.25, random_state=0)


pca = PCA(n_components=6)

pca.fit(X_train, y_test)
## Train
PCA_X_train = pca.transform(X_train)

## Test
PCA_X_test = pca.transform(X_test)

In [27]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[1.59132602e+13 1.47215292e+11 1.11923187e+11 1.12251137e+10
 2.63431948e+09 1.09330381e+08]
[9.83127330e-01 9.09501728e-03 6.91465750e-03 6.93491837e-04
 1.62749270e-04 6.75447294e-06]
[0.98312733 0.99222235 0.999137   0.9998305  0.99999325 1.        ]


In [28]:
PCA_model_avocado.fit(PCA_X_train, y_train)
pred_train = PCA_model_avocado.predict(PCA_X_train)
pred_test = PCA_model_avocado.predict(PCA_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.1541682995864888
MSE_Test: 0.15300610412254645


### Con variables categoricas

In [29]:
ohe = OneHotEncoder().fit(categorical_data)
categorical_bin = ohe.transform(categorical_data)
categorical_bin = pd.DataFrame(categorical_bin.toarray(), columns=ohe.get_feature_names())
categorical_bin.head()

,x0_conventional,x0_organic,x1_Albany,x1_Atlanta,x1_BaltimoreWashington,x1_Boise,x1_Boston,x1_BuffaloRochester,x1_California,x1_Charlotte,...,x1_SouthCarolina,x1_SouthCentral,x1_Southeast,x1_Spokane,x1_StLouis,x1_Syracuse,x1_Tampa,x1_TotalUS,x1_West,x1_WestTexNewMexico
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
PCA_model_avocado_cat = LinearRegression()

In [31]:
X_avocado_ohe = pd.concat([X_avocado,categorical_bin], axis=1)
X_avocado_ohe

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,x0_conventional,x0_organic,...,x1_SouthCarolina,x1_SouthCentral,x1_Southeast,x1_Spokane,x1_StLouis,x1_Syracuse,x1_Tampa,x1_TotalUS,x1_West,x1_WestTexNewMexico
0,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18245,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18246,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18247,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_avocado_ohe, y_avocado, test_size=0.25, random_state=0)


pca = PCA(n_components=6)

pca.fit(X_train, y_train)
## Train
PCA_X_train = pca.transform(X_train)

## Test
PCA_X_test = pca.transform(X_test)


In [33]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[1.59132602e+13 1.47215292e+11 1.11923187e+11 1.12251137e+10
 2.63431948e+09 1.09330381e+08]
[9.83127330e-01 9.09501728e-03 6.91465750e-03 6.93491837e-04
 1.62749270e-04 6.75447294e-06]
[0.98312733 0.99222235 0.999137   0.9998305  0.99999325 1.        ]


In [34]:
PCA_model_avocado_cat.fit(PCA_X_train, y_train)
pred_train = PCA_model_avocado_cat.predict(PCA_X_train)
pred_test = PCA_model_avocado_cat.predict(PCA_X_test)

print("MSE_Train:",mean_squared_error(pred_train, y_train))
print("MSE_Test:",mean_squared_error(pred_test, y_test))

MSE_Train: 0.15416829958679562
MSE_Test: 0.1530061041226993


# Análisis
*  **Abalon Dataset**

Se puede ver en los resultados de porcentajes de varianza de la matriz de covarianza para cada una de las componentes de PCA en cada uno de los casos (con y sin variables categóricas) una leve diferencia, menor al 10% en las componentes más significativas. Esta diferencia porcentual menor tiene relación a la forma que se computa PCA, haciendo que sea dificil para el algoritmo entregar un valor o un peso para poder relacionarlo con los otros atributos del dataset, entregando resultados que no tienen mucha información y terminan por añadir solamente ruido al dataset. Adicionalmente, esta diferencia puede ser relacionada con la inclusión de nuevas componentes al agregar más dimensiones con el OHE (One-Hot Encoding), que entrega tres columnas para una columna categorica (columna `Sex` con valores `M, F, I`).

También, se puede ver que no existen diferencias significativas en los errores cuadráticos medios de los modelos aplicados a los datos con PCA y sin PCA, que están relacionados con la metodología de cómputo de PCA y la introducción de ruidos al agregar columnas adicionales.

Adicionalmente, se puede ver que el método MCA, que funciona mejor para datos categoricos, tampoco nos entrega resultados mucho mejores, debido a la naturaleza de nuestros datos, que correponde a una mixta (datos continuos y datos categoricos), lo que nos hace pensar que un método que aplique PCA y MCA sobre nuestros atributos pueda entregar resultados más significativos

* **Avocado Dataset**

Del mismo modo que para el Abalon dataset, las diferencias que se tuvieron fueron no significativas, lo que nos confirma lo propuesto anteriormente, en especial a la diferencia que existe debido a la inclusión de nuevas columnas al dataset, y por lo tanto, nuevas componentes a PCA.

* **Dataset Sintéticos**

Para los datos sínteticos se trabaja con datos de naturaleza únicamente categorica, para analizar mejor el comportamiento de PCA dentro de estas condiciones. En este dataset las categorias estan representadas por números enteros en primera instancia y que posteriormente se codifican utilizando OHE. En el primer caso, si bien los datos tienen un peso asociado a su valor, este valor no entrega un significado al modelo, por lo que nuevamente el cálculo de PCA se verá afectado por estas condiciones y los resultados que entrega no muestran una correlación entre los distintos atributos que existen dentro de los datos. Esta condición se ve mucho más claramente al hacer el OHE, debido a que nuestros atributos de ser cerca de 20 a ser 160 aproximadamente, y por lo tanto nuestra cantidad de componentes también aumenta drásticamente. Por esto último se modifican el porcentaje de componentes que se le solicitarán a PCA respecto a la cantidad de atributos, pasando de ser un 50% para el caso sin OHE a 25% con OHE. Este cambio confirma nuevamente la introducción de ruido dentro del dataset, donde una disminución porcentual significativa dentro de las componentes de PCA, entrega un porcentaje de varianza acumulado similar a la que se obtenía sin introducir OHE


# Desafío Kaggle (Mercado Libre) - Desarrollo
*Objetivo: El problema propuesto consiste en predecir la categoría a la que pertenece un anunció a partir de su título*

In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier

try:
    from imblearn.over_sampling import SMOTE
except ImportError:
    os.system('pip install -U imbalanced-learn')
    from imblearn.over_sampling import SMOTE
    
try:
    import nltk
except ImportError:
    os.system('pip install nltk')
    import nltk
#nltk.download('all-corpora')
#nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /users/cnunez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /users/cnunez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Comenzamos cargando el dataset de la competencia:

In [36]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('x_test.csv')

## Definicion Y
A continuacion, definimos la variable `Y` como el conjunto de etiquetas:

In [37]:
Y = df_train.category

## Pre-procesamiento de texto

Dado que la columna títulos corresponde a un texto, debemos procesar el texto por medio de una tokenización, lematización, y Stemming. Para ello, utilizamod la librería `nltk`.

In [38]:
import spacy
import string
import re
#nlp_es = spacy.load('es_core_news_sm')

import spacy
#nlp = spacy.load("es_core_news_sm", disable=['parser', 'tagger', 'ner'])
stops_es = stopwords.words("spanish")
nlp = spacy.load('es_core_news_sm')

import unidecode

In [39]:
def LemNormalizeSpacy(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text_lower = text.lower().translate(remove_punct_dict)
    doc = nlp(text_lower)
    solo_lemmas=[]
    for sent in doc.sents:
        for token in sent:
            lemma = token.lemma_.strip()  # get the lemma
            lemma = unidecode.unidecode(lemma)  # Remove accents
            lemma = re.sub(r'\d+', '',lemma)  # Remove numbers
            lemma = re.sub(r'_', '', lemma)
            if lemma and lemma not in stops_es:
                solo_lemmas.append(lemma)
    return solo_lemmas

In [40]:
tokenizer = TfidfVectorizer().build_tokenizer()
stemmer = SnowballStemmer('spanish')
lemmatizer = WordNetLemmatizer()

remove_stops_here = True

def my_pre_processer(text):
    results = []
    for token in tokenizer(text):
        clean_token = token.lower().strip().strip('-').strip('_')
        if remove_stops_here and (clean_token in stopwords.words('spanish')):
          continue
        #token_pro = stemmer.stem(clean_token) #podemos probar stemming en vez de lematizacion
        token_pro = lemmatizer.lemmatize(clean_token) 
        if len(token_pro) > 2 and not token_pro[0].isdigit(): #elimina palabra largo menor a 2
            results.append(token_pro)
    return results

Un ejemplo de como sería el pre-procesamiento es:

In [41]:
print(df_train.title.values[1])
print(my_pre_processer(df_train.title.values[1]))

Kit 4 Bujias Ngk Ford F-100 3.0 Bp5efs
['kit', 'bujias', 'ngk', 'ford', 'bp5efs']


In [42]:
LemNormalizeSpacy(df_train.title.values[1])

['kit', 'bujias', 'ngk', 'ford', 'f', 'bpefs']

## Vectorización de Texto

Ahora toca una de las partes más importantes, que es vectorizar la representación del texto para cada título del dataset. Para ello utilizaremos la clásica técnica `Tf-idf`.

In [43]:
max_df=0.9
min_df=5
max_features=100000

vectorizer = TfidfVectorizer(tokenizer=LemNormalizeSpacy, min_df=min_df, max_df=max_df, 
                             max_features=max_features, use_idf=True, smooth_idf=True, 
                             strip_accents='unicode', analyzer = 'word')

In [44]:
start_time = time()
vectorizer.fit(df_train.title)
X = vectorizer.transform(df_train.title)
end_time = time()

print('La operación demoró {}[s]'.format(end_time - start_time))

La operación demoró 1509.8772060871124[s]


## Balanceamiento del dataset - SMOTE

Dado que el dataset puede estar mal condicionado o desbalanceado, utilizaremos la técnica de `SMOTE` para generar un sample balanceado de datos.

In [45]:
oversample = SMOTE(random_state = 42, n_jobs = -1)

In [46]:
start_time = time()
X,Y = oversample.fit_resample(X, Y)
end_time = time()

print('La operación demoró {}[s]'.format(end_time - start_time))

La operación demoró 6.832873106002808[s]


## Diseño del clasificador

Para cumplir con nuestro objetivo, utilizaremos un regreso logistico entrenaddo con la tecnica OVR para clasificar las distintas categoricas de los productos. Por otro lado, se utilizará la implementación `LogisticRegressionCV` para incluir cross-validation en el entrenamiento.

**Declaimer**: Esta parte puede demorar muchos más en comparación al resto del código.

In [47]:
base = LogisticRegressionCV(cv=10, random_state=42, max_iter=10000, solver='newton-cg', n_jobs=-1, verbose=1)
clf = OneVsRestClassifier(base)

start_time = time()
clf.fit(X,Y)
end_time = time()

print('La operación demoró {}[s]'.format(end_time - start_time))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    7.9s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    5.3s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    4.6s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    7.9s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parall

La operación demoró 409.74054312705994[s]


## Predicción de datos

Comenzaremos vectorizando los títulos que debemos predecir:

In [48]:
start_time = time()
vector_titles_submission = vectorizer.transform(df_test.title)
end_time = time()

print('La operación demoró {}[s]'.format(end_time - start_time))

La operación demoró 510.38388991355896[s]


Luego realizamos la predicción sobre el set de datos de la competencia:

In [49]:
y_pred_submission = clf.predict(vector_titles_submission)
df_test.Predicted = y_pred_submission

df_test = df_test.drop(columns=['title'])

Finalmente guardamos el dataset final con las etiquetas para realizar el submission:

In [50]:
df_test.to_csv('last_submission.csv', index=False, header=True)